In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="JdViCZkE1bnhWcRhwrM1")
project = rf.workspace("deep-learning-mvdfy").project("deep-learning-khbes")
version = project.version(3)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Deep-learning-3 in yolov8:: 100%|██████████| 212/212 [00:00<00:00, 5189.00it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [1]:
pip install ultralytics optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00


# Import Libraries

In [3]:
import torch
import yaml
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path
from typing import Tuple, Optional, Dict, Any, List

# YOLO Image Preprocessor

This class, `YOLOImagePreprocessor`, is designed to preprocess images for use with a YOLO (You Only Look Once) object detection model. The preprocessing involves loading images, cropping out any black borders, resizing the image while maintaining the aspect ratio, and padding the image to the target size. This ensures the images are uniform and ready for input into the model.

### Class: `YOLOImagePreprocessor`

- **`__init__` Method**: Initializes the preprocessor with a target image size (default is 640x640) and a padding color of (114, 114, 114) used to fill empty spaces after resizing.

- **`process_image` Method**:
  - This method processes an individual image:
    - It reads the image from the given file path and converts the color format from BGR (OpenCV default) to RGB.
    - It creates a mask to detect non-black areas and crops the image to remove unnecessary black regions.
    - After cropping, the image is resized to fit the target size, maintaining its original aspect ratio.
    - The image is padded with the specified padding color to match the target size exactly, ensuring it is ready for input into the YOLO model.

- **`process_dataset` Method**:
  - This method processes a whole dataset:
    - It iterates through all the images in the provided dataset directory and applies the `process_image` method to each image.
    - The processed images are saved in the output directory, and a record of the original and processed image sizes is kept in a dictionary.

### Error Handling

- If an image cannot be loaded or is completely black, an error is raised, and the image is skipped.
- Any issues during dataset processing are caught, and an error message is printed, but the overall process continues.

This preprocessing pipeline ensures that images fed into the YOLO model are consistent in size and format, with unnecessary areas removed and all images scaled appropriately for the model.


In [4]:


class YOLOImagePreprocessor:

    def __init__(self, target_size: Tuple[int, int] = (640, 640)):
        self.target_size = target_size
        self.padding_color = (114, 114, 114)

    def process_image(self, image_path: str) -> Optional[np.ndarray]:

        try:
            image = cv2.imread(str(image_path))
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            mask = np.any(image != [0, 0, 0], axis=2)
            rows = np.any(mask, axis=1)
            cols = np.any(mask, axis=0)

            if not np.any(rows) or not np.any(cols):
                raise ValueError(f"Image {image_path} appears to be completely black")

            y1, y2 = np.where(rows)[0][[0, -1]]
            x1, x2 = np.where(cols)[0][[0, -1]]

            cropped = image[y1:y2+1, x1:x2+1]

            h, w = cropped.shape[:2]
            scale = min(self.target_size[0]/w, self.target_size[1]/h)
            new_w = int(w * scale)
            new_h = int(h * scale)

            resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_AREA)

            canvas = np.ones((self.target_size[0], self.target_size[1], 3),
                           dtype=np.uint8) * self.padding_color[0]

            pad_x = (self.target_size[1] - new_w) // 2
            pad_y = (self.target_size[0] - new_h) // 2

            canvas[pad_y:pad_y+new_h, pad_x:pad_x+new_w] = resized

            return canvas

        except Exception as e:
            print(f"Error preprocessing image {image_path}: {str(e)}")
            return None

    def process_dataset(self, dataset_path: Path, output_path: Path) -> Dict[str, Dict[str, int]]:
        output_path.mkdir(exist_ok=True, parents=True)
        transformations = {}

        for img_path in dataset_path.glob('*.jpg'):
            try:
                processed = self.process_image(str(img_path))
                if processed is not None:
                    output_img_path = output_path / img_path.name
                    cv2.imwrite(str(output_img_path), cv2.cvtColor(processed, cv2.COLOR_RGB2BGR))

                    transformations[img_path.name] = {
                        'original_size': tuple(cv2.imread(str(img_path)).shape[:2]),
                        'processed_size': processed.shape[:2]
                    }
            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")

        return transformations

# YOLO Trainer

The `YOLOTrainer` class is designed to train a YOLO object detection model on a given dataset. It facilitates dataset preparation, model training with customizable hyperparameters, and saving the trained model.

### Class: `YOLOTrainer`

#### `__init__` Method
```python
def __init__(self, dataset_path: str, model_path: str = 'yolov8n.pt', save_dir: str = 'saved_models'):


In [ ]:
class YOLOTrainer:
    def __init__(self,
                 dataset_path: str,
                 model_path: str = 'yolov8n.pt',
                 save_dir: str = 'saved_models'):
        self.dataset_path = dataset_path
        self.model_path = model_path
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True, parents=True)
        self.preprocessor = YOLOImagePreprocessor(target_size=(640, 640))

        with open(dataset_path, 'r') as f:
            self.dataset_config = yaml.safe_load(f)

        print("Dataset Configuration:")
        print(yaml.dump(self.dataset_config, default_flow_style=False))

        self.hyperparameters = {
            'lr0': 0.01,
            'lrf': 0.05,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3,
            'warmup_momentum': 0.8,
            'batch_size': 8,
            'mosaic': 0.5
        }

    def prepare_dataset(self) -> Tuple[Dict[str, Dict[str, int]], Dict[str, Dict[str, int]]]:
        """
        Prepare and validate the dataset before training.
        """
        train_path = Path(self.dataset_config.get('train', ''))
        val_path = Path(self.dataset_config.get('val', ''))

        train_transforms = self.preprocessor.process_dataset(
            train_path,
            train_path.parent / 'processed_train'
        )
        val_transforms = self.preprocessor.process_dataset(
            val_path,
            val_path.parent / 'processed_val'
        )

        return train_transforms, val_transforms

    def train_model(self,
               save_name: str = 'trained_model',
               epochs: int = 100) -> Dict[str, Any]:
      """
      Train model with fixed hyperparameters, custom number of classes,
      and frozen backbone layers.
      """
      model = YOLO(self.model_path)
      if hasattr(model, 'model'):
          for param in model.model.parameters():
              param.requires_grad = False

          if hasattr(model.model, 'model'):
              last_layer = model.model.model[-1]
          else:
              last_layer = model.model[-1]

          for param in last_layer.parameters():
              param.requires_grad = True

          last_layer.nc = len(self.dataset_config['names'])

      save_dir = self.save_dir / save_name
      save_dir.mkdir(exist_ok=True)

      args = {
          'data': self.dataset_path,
          'epochs': epochs,
          'batch': self.hyperparameters['batch_size'],
          'lr0': self.hyperparameters['lr0'],
          'lrf': self.hyperparameters['lrf'],
          'momentum': self.hyperparameters['momentum'],
          'weight_decay': self.hyperparameters['weight_decay'],
          'warmup_epochs': self.hyperparameters['warmup_epochs'],
          'warmup_momentum': self.hyperparameters['warmup_momentum'],
          'mosaic': self.hyperparameters['mosaic'],
          'project': str(self.save_dir),
          'name': save_name,
          'exist_ok': True
      }

      results = model.train(**args)

      model_save_path = save_dir / 'weights'
      model_save_path.mkdir(exist_ok=True)

      model.save(str(model_save_path / 'best.pt'))
      model.save(str(model_save_path / 'last.pt'))

      with open(save_dir / 'training_results.yaml', 'w') as f:
          yaml.dump(results.results_dict, f)

      return results.results_dict


def main():
    """
    Main execution function.
    """
    dataset_path = '/content/Deep-learning-3/data.yaml'
    save_dir = '/content/drive/MyDrive/hemanth'
    epochs = 250

    trainer = YOLOTrainer(
        dataset_path=dataset_path,
        model_path='yolov8n.pt',
        save_dir=save_dir
    )

    print("\nTraining model...")
    results = trainer.train_model(
        save_name='trained_model',
        epochs=epochs
    )


    print("\nTraining completed successfully!")
    print(f"Model and results saved in: {save_dir}/trained_model/")

if __name__ == '__main__':
    main()

Dataset Configuration:
names:
- building
- road
- tree
nc: 3
roboflow:
  license: CC BY 4.0
  project: deep-learning-khbes
  url: https://universe.roboflow.com/deep-learning-mvdfy/deep-learning-khbes/dataset/3
  version: 3
  workspace: deep-learning-mvdfy
test: ../test/images
train: ../train/images
val: ../valid/images


Training model...


100%|██████████| 6.25M/6.25M [00:00<00:00, 109MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Deep-learning-3/data.yaml, epochs=250, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/hemanth, name=trained_model, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

100%|██████████| 755k/755k [00:00<00:00, 19.8MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     54/250         0G     0.8723      1.485      1.369         23        640: 100%|██████████| 10/10 [01:02<00:00,  6.27s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.56s/it]

                   all         10         17      0.397      0.426      0.304      0.141



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     55/250         0G      0.879      1.459      1.422         25        640:  20%|██        | 2/10 [00:18<00:50,  6.33s/it]

# Model Architecture

In [10]:
import torch
from ultralytics import YOLO

model = YOLO("yolov8n")

print(model.model)
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s